## Setup mysql database

```bash
brew install mysql
brew services start mysql
brew services stop mysql
brew services list
mysql_secure_installation
mysql --version
mysql -u root -p
```

Put this into a file `sudo nano /etc/my.cnf`:
```bash
[client]
local_infile=1

[mysqld]
local_infile=1
```

```bash
brew services restart mysql
```

CREATE DATABASE database_name;





## Setup postgres SQL database

Install Postgres via website or via brew:
```bash
brew install postgresql
brew services start postgresql
psql --version
```

Add psql to path to ~/.zshrc:
```bash
nano ~/.zshrc
export PATH="/Library/PostgreSQL/16/bin:$PATH"
```

Now login as postgres user:
```bash
psql -U postgres
```

Crate a new database:
```bash
CREATE DATABASE hello_world_db;
```

List all databases:
```bash
\l
```

Now one can use `SQLTools` extension in VSCode, and connect to database.



# Create table and copy data from csv file

```
\copy employees(id, first_name, last_name, email, department_id) FROM '/Users/nenadbozinovic/Documents/blog/nbs/tips/015_Cloud/aws/certificates/data_engineer/employees.csv' DELIMITER ',' CSV HEADER;
```


## Constraint in SQL

Foreign key (FK) constraint can be added to a table, for example below, `Orders` table must have a `CustomerID` that matches `Customer(CustomerID)`. This ensures data integrity, so we don't have nonsense Orders that have no matching Customer.

```sql
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(100)
);

CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    OrderDate DATE,
    CustomerID INT,
    CONSTRAINT FK_CustomerOrder FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
```


## Sample multipart upload calls

For this example, assume that you are generating a multipart upload for a 100 GB file. In this case, you would have the following API calls for the entire process. There would be a total of 1002 API calls.

A CreateMultipartUpload call to start the process.
1000 individual UploadPart calls, each uploading a part of 100 MB, for a total size of 100 GB.
A CompleteMultipartUpload call to finish the process.

```bash
aws s3 cp your-file.txt s3://your-bucket-name/your-file.txt --storage-class DEEP_ARCHIVE
```

## Exercises

hint: `avg(Survived)`  is the same as `(COUNT(CASE WHEN Survived = 1 THEN 1 END) * 1.0 / count(*))` since Survived is 0 or 1.

```sql
SELECT * FROM titanic LIMIT 10;
select (COUNT(CASE WHEN Survived = 1 THEN 1 END) * 1.0 / count(*)) as overall_rate FROM titanic;
select avg(Survived) as women_children_rate FROM titanic WHERE (Sex="female" OR Age<=12);
select avg(Survived) as others_rate FROM titanic WHERE NOT (Sex="female" OR Age<=12);
```

```sql
SELECT 
    Pclass, 
    AVG(Survived) AS survival_rate
FROM 
    titanic
GROUP BY 
    Pclass
ORDER BY 
    Pclass;
```

```sql
select
    Products.ProductName,
    Suppliers.CompanyName
from
    Products
left join
    Suppliers
on
    Products.SupplierID = Suppliers.SupplierID
order by
    Products.ProductName;
```